In [41]:
import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

from math import sin, radians, sqrt

import ipywidgets as widgets

In [42]:
set_defaults(axes=True, grid=True, axes0=True, timeit=False, cad_width=640, height=800)

set_sidecar("clamp", init=True)
replay_box = widgets.Checkbox(False, description='Enable Replay')
out = widgets.Output()
display(replay_box)
display(out)

@out.capture()
def changed(e):
    if e["new"]:
        enable_replay(warning=False)
    else:
        disable_replay()

replay_box.observe(changed, names='value')

Checkbox(value=False, description='Enable Replay')

Output()

In [43]:
tube_d = 4
screw_d = 10-0.5
screw_h = 10
filament_d = 1.75+0.3
thickness = 2
flange = 1

In [45]:
fake_screw = screw_d/sqrt(2)

cutter = (
    cq.Workplane("XY")
    .move(0, fake_screw/2)
    .rect(tube_d+thickness*4+flange*2, thickness*6, centered=(True,False))
    .mirrorX()
    .extrude(thickness*3)
    .faces("<Y[1]").workplane(centerOption="CenterOfMass")
    .rect(tube_d+thickness*2+flange*2, thickness)
    .extrude(fake_screw/2)
    .faces("<Z[1]").workplane(centerOption="CenterOfMass")
    .rect(tube_d+thickness*2-0.5, fake_screw/2)
    .extrude(thickness)
)

replay(cutter)

Use the multi select box below to select one or more steps you want to examine


In [46]:
clamp = (
    cq.Workplane("XY")
    ################
    .rect(fake_screw, fake_screw)
    .extrude(-screw_h/2)
    .faces("<Z")
    .wires()
    .toPending()
    .workplane(offset=screw_h/2)
    .circle(fake_screw/2)
    .loft()
    ################
    .faces(">Z")
    .workplane()
    .circle(tube_d/2+thickness*2+flange)
    .extrude(thickness)
    .faces(">Z")
    .workplane()
    .circle(tube_d/2+thickness*2+flange)
    .circle(tube_d/2+thickness+flange)
    .extrude(thickness)
    .faces(">Z")
    .workplane()
    .circle(tube_d/2+thickness*2+flange)
    .circle(tube_d/2+thickness)
    .extrude(thickness)
    ################
    .cut(cutter)
    ################
    .faces("<Z")
    .workplane()
    .cboreHole(filament_d, tube_d, 2)
    .faces("<Z[2]")
    .workplane()
    .cskHole(filament_d, filament_d+1, 45)
)


replay(clamp)

Use the multi select box below to select one or more steps you want to examine


In [47]:
helper = (
    cq.Workplane("XY")
    .circle(tube_d/2+thickness+flange)
    .extrude(thickness)
    .faces(">Z").workplane()
    .circle(tube_d/2+thickness)
    .extrude(thickness+5)
    .faces(">Z").workplane()
    .hole(tube_d)
)

replay(helper)

Use the multi select box below to select one or more steps you want to examine


In [48]:
show(clamp+helper.translate((20, 0, 0)))
exporters.export(clamp, '/tmp/clamp.stl')
exporters.export(helper, '/tmp/clamp_helper.stl')

Done, using side car 'Clamp'


In [39]:
x = cq.Workplane("XY").sketch()\
    .regularPolygon(5/sqrt(3), 6)\
    .finalize()\
    .extrude(10)

x = cq.Workplane("XY").circle(15/2).extrude(10).cut(x)

replay(x)
exporters.export(x, '/tmp/x.stl')

Use the multi select box below to select one or more steps you want to examine
